In [ ]:
# Run this in powershell to start qdrant
# docker run -p 6333:6333 -v D:\qdrant_data:/qdrant/storage qdrant/qdrant

# Create collection and load one document

In [32]:
import torch
test_tensor = torch.load(r"data\colpali_embeddings\0000320193-23-000005.pt")
test_tensor[0].shape

# It needs to be 

C:\Users\Kenta Sakai\AppData\Local\Temp\ipykernel_9984\2227660810.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test_tensor = torch.load(r"data\colpali_embeddings\0000

torch.Size([1030, 128])

In [ ]:
from qdrant_client import QdrantClient, models
import config

client = QdrantClient("http://localhost:6333") # Connect to existing Qdrant instance

# https://python-client.qdrant.tech/qdrant_client.http.models.models.html?highlight=sparsevector#qdrant_client.http.models.models.SparseVectorParams
# sparse_vectors_config = {
#     "splade": models.SparseVectorParams()
# }

# https://qdrant.tech/documentation/concepts/vectors/
vectors_config = {
    "colpali": models.VectorParams(
        size=128,
        distance=models.Distance.COSINE,
        multivector_config=models.MultiVectorConfig(
            comparator=models.MultiVectorComparator.MAX_SIM
        )
    )
}

# client.create_collection(
#     collection_name=config.COLLECTION_NAME,
#     vectors_config=vectors_config,
# )

In [33]:
import uuid

# Upsert the point into the Qdrant collection
client.upsert(
    collection_name=config.COLLECTION_NAME,
    points=[
        models.PointStruct(
            id=1, 
            vector={
                "colpali": test_tensor[0].tolist()  # Provide the vector as a flat list
            }
        ),
        models.PointStruct(
            id=2,  
            vector={
                "colpali": test_tensor[1].tolist()  # Provide the vector as a flat list
            }
        ),
        models.PointStruct(
            id=3,  
            vector={
                "colpali": test_tensor[2].tolist()  # Provide the vector as a flat list
            }
        ),
        models.PointStruct(
            id=4,  
            vector={
                "colpali": test_tensor[3].tolist()  # Provide the vector as a flat list
            }
        ),
    ],
)




UpdateResult(operation_id=7, status=<UpdateStatus.COMPLETED: 'completed'>)

In [31]:
# How to add payload
client.set_payload(
    collection_name=config.COLLECTION_NAME,
    payload={
        "page": 0,
        "company": "Apple, Inc.",
    },
    points=["0d8885f2-d208-40f8-bf40-b539719c20b2"],
)

UpdateResult(operation_id=6, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
# How to query
# https://qdrant.tech/documentation/concepts/search/

client.query_points(
    collection_name= config.COLLECTION_NAME,
    query=[0.2, 0.1, 0.9, 0.7], 
    using="colpali"
)